In [4]:
import gradio as gr
import torch
from Utilities import config
from Utilities.model import YOLOv3
from Utilities.transforms import test_transforms
from Utilities.utils import cells_to_bboxes, non_max_suppression, plot_image

In [6]:
model = YOLOv3.load_from_checkpoint( "Store/epoch=39-step=16560.ckpt", map_location=torch.device("cpu"))
# model()

In [ ]:
model = YOLOv3(num_classes=config.NUM_CLASSES)

# model.load_state_dict(torch.load("Store/yolov3.pth", map_location=torch.device("cpu")))
model.load_from_checkpoint(
    "Store/epoch=39-step=16560.ckpt", map_location=torch.device("cpu"))

YOLOv3(
  (layers): ModuleList(
    (0): CNNBlock(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky): LeakyReLU(negative_slope=0.1)
    )
    (1): CNNBlock(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky): LeakyReLU(negative_slope=0.1)
    )
    (2): ResidualBlock(
      (layers): ModuleList(
        (0): Sequential(
          (0): CNNBlock(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (leaky): LeakyReLU(negative_slope=0.1)
          )
          (1): CNNBlock(
            (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=

In [7]:
model.layers[-1]

ScalePrediction(
  (pred): Sequential(
    (0): CNNBlock(
      (conv): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky): LeakyReLU(negative_slope=0.1)
    )
    (1): CNNBlock(
      (conv): Conv2d(256, 75, kernel_size=(1, 1), stride=(1, 1))
      (bn): BatchNorm2d(75, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky): LeakyReLU(negative_slope=0.1)
    )
  )
)

In [3]:
torch.save(model.state_dict(), "Store/yolov3.pth")

In [5]:
def plot_bbox(input_img, thresh=0.5, iou_thresh=0.6, anchors=model.scaled_anchors):
    input_img = test_transforms(image=input_img)["image"]
    input_img = input_img.unsqueeze(0)
    model.eval()
    with torch.no_grad():
        out = model(input_img)
        # bboxes = [[] for _ in range(input_img.shape[0])]
        # bboxes = []
        for i in range(3):
            batch_size, A, S, _, _ = out[i].shape
            anchor = anchors[i]
            boxes_scale_i = cells_to_bboxes(out[i], anchor, S=S, is_preds=True)
            bboxes = boxes_scale_i[0]
            # for idx, (box) in enumerate(boxes_scale_i):
            #     bboxes[idx] += box

    # for i in range(batch_size // 4):
    nms_boxes = non_max_suppression(
        bboxes,
        iou_threshold=iou_thresh,
        threshold=thresh,
        box_format="midpoint",
    )
    fig = plot_image(input_img[0].permute(1, 2, 0).detach().cpu(), nms_boxes)
    return fig

In [27]:
out_0 = model(torch.zeros(1, 3, 416, 416))[0]

In [30]:
out_0[out_0.argmax(-1)]

IndexError: index 19 is out of bounds for dimension 0 with size 1

In [32]:
out_0.size()

torch.Size([1, 3, 13, 13, 25])

In [34]:
# Assuming 'out' is your tensor of size (1, 3, 13, 13, 25)
# and you want to get the 25 elements for the highest score for each (1, 3, 13, 13) element

# Reshape the tensor to collapse the last two dimensions
reshaped_out = out_0.view(1, 3, 13, 13, -1)

# Get the highest score index (0th index) along the last dimension
highest_score_indices = torch.argmax(reshaped_out[..., 0], dim=-1)

# Gather the 25 elements from the last dimension based on the highest score indices
highest_score_elements = torch.gather(reshaped_out, dim=-1, index=highest_score_indices.unsqueeze(-1))

# The shape of highest_score_elements will be (1, 3, 13, 13, 1, 25), so squeeze to remove the extra dimension
highest_score_elements = highest_score_elements.squeeze(-2)

# Now highest_score_elements contains the 25 elements for the highest score for each (1, 3, 13, 13) element
print(highest_score_elements)

RuntimeError: Index tensor must have the same number of dimensions as input tensor

In [49]:
# Step 1: Extract objectness scores
objectness_scores = out_0[..., 0]

# Step 2: Get the index of the highest objectness score
max_score_index = torch.argmax(objectness_scores)

In [55]:
# Extract the 25 elements in the last dimension for the corresponding index
selected_elements = out_0[0, :, max_score_index // 13, max_score_index % 13, 1:]

In [56]:
selected_elements.shape

torch.Size([3, 24])

In [63]:
torch.argmax(out_0, dim=(1,2,3)).shape

TypeError: argmax(): argument 'dim' must be int, not tuple

In [68]:
max_obj_arg = objectness_scores.argmax()

In [71]:
max_obj_arg_onehot = torch.zeros(objectness_scores.flatten().shape[0])

In [72]:
max_obj_arg_onehot[max_obj_arg] = 1

In [82]:
max_obj_arg_onehot = max_obj_arg_onehot.reshape_as(objectness_scores,).int()

In [84]:
max_obj_arg_onehot.shape

torch.Size([1, 3, 13, 13])

In [94]:
selected_elements = out_0[max_obj_arg_onehot==1]

In [95]:
selected_elements

tensor([[-1.1751,  0.1118,  1.0949, -0.7356, -0.8127, -3.3133,  1.3863, -1.5563,
          0.0267,  1.3461, -2.8634, -0.6618, -1.9784, -1.4624, -3.6695, -3.3399,
         -2.9955, -1.8513,  1.0410,  3.3047,  3.7847, -3.9926, -4.6339, -0.9043,
         -1.7867]], grad_fn=<IndexBackward0>)

In [140]:
def return_top_objectness_class_preds(model, input_img):
    out = model(input_img)[-1]
    
    # Step 1: Extract objectness scores
    objectness_scores = out[..., 0]

    # Step 2: Get the index of the highest objectness score
    max_obj_arg = torch.argmax(objectness_scores)

    max_obj_arg_onehot = torch.zeros(objectness_scores.flatten().shape[0])
    max_obj_arg_onehot[max_obj_arg] = 1

    max_obj_arg_onehot = max_obj_arg_onehot.reshape_as(objectness_scores,).int()    

    selected_elements = out[max_obj_arg_onehot==1]
    selected_elements = selected_elements[:, 5:]

    return selected_elements

In [142]:
return_top_objectness_class_preds(model, torch.zeros(1, 3, 416, 416)).shape

torch.Size([1, 20])

In [116]:
import pytorch_lightning as pl

input_img = torch.zeros(1, 3, 416, 416)
org_img = input_img

def return_top_objectness_class_preds(model, input_img):
    out = model(input_img)[0]

    # Step 1: Extract objectness scores
    objectness_scores = out[..., 0]

    # Step 2: Get the index of the highest objectness score
    max_obj_arg = torch.argmax(objectness_scores)

    max_obj_arg_onehot = torch.zeros(objectness_scores.flatten().shape[0])
    max_obj_arg_onehot[max_obj_arg] = 1

    max_obj_arg_onehot = max_obj_arg_onehot.reshape_as(
        objectness_scores,
    ).int()

    selected_elements = out[max_obj_arg_onehot == 1]
    selected_elements = selected_elements[:, 5:]

    return selected_elements


class TopObjectnessClassPreds(pl.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, x):
        return return_top_objectness_class_preds(self.model, x)


TopObjectnessClassPredsObj = TopObjectnessClassPreds(model)

In [117]:
model.layers[-5]

CNNBlock(
  (conv): Conv2d(384, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (leaky): LeakyReLU(negative_slope=0.1)
)

In [1]:
model.layers

NameError: name 'model' is not defined

In [125]:
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

cam = GradCAM(
        model=TopObjectnessClassPredsObj,
        target_layers=[model.layers[-1]],
    )
grayscale_cam = cam(input_tensor=input_img, targets=None)
grayscale_cam = grayscale_cam[0, :]

visualization = show_cam_on_image(
    org_img / 255,
    grayscale_cam,
    use_rgb=True,
    image_weight=0.5,
)

AxisError: axis 2 is out of bounds for array of dimension 0

In [8]:
model.layers[-1]

ScalePrediction(
  (pred): Sequential(
    (0): CNNBlock(
      (conv): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky): LeakyReLU(negative_slope=0.1)
    )
    (1): CNNBlock(
      (conv): Conv2d(256, 75, kernel_size=(1, 1), stride=(1, 1))
      (bn): BatchNorm2d(75, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky): LeakyReLU(negative_slope=0.1)
    )
  )
)

In [24]:
for child in model.layers[-1].named_children():
    print(child[0])

pred


In [27]:
next(model.layers[-1].named_children())[0]

'pred'

In [31]:
for layer_idx, layer in enumerate(model.layers):
    try:
        if next(layer.named_children())[0] == 'pred':
            print(layer_idx)
            # print(layer)
    except:
        pass

15
22
29


In [32]:
model.layers[15]

ScalePrediction(
  (pred): Sequential(
    (0): CNNBlock(
      (conv): Conv2d(512, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky): LeakyReLU(negative_slope=0.1)
    )
    (1): CNNBlock(
      (conv): Conv2d(1024, 75, kernel_size=(1, 1), stride=(1, 1))
      (bn): BatchNorm2d(75, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky): LeakyReLU(negative_slope=0.1)
    )
  )
)

In [33]:
model.layers[22]

ScalePrediction(
  (pred): Sequential(
    (0): CNNBlock(
      (conv): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky): LeakyReLU(negative_slope=0.1)
    )
    (1): CNNBlock(
      (conv): Conv2d(512, 75, kernel_size=(1, 1), stride=(1, 1))
      (bn): BatchNorm2d(75, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky): LeakyReLU(negative_slope=0.1)
    )
  )
)